In [ ]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

In [ ]:
# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

In [ ]:
# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [ ]:
# Given order
order = [
    "Актуализация", "Внебаланс (new)", "ДИЗ", "ДИЗ (возражения)", "ДИЗ (чередование)",
    "Коллекшн ранний сбор", "Матрица мотиваторов", "Матрица мотиваторов (распознавание эмоций)",
    "Матрица мотиваторов Хард. Ранний сбор", "Матрица мотиваторов Хард. Ранний сбор (кредитные карты)",
    "Матрица мотивации. Выезд", "Мультидоговоры. Универсальный", "Неконтактные", "Неконтактные (внебаланс)",
    "Неконтактные (поздний сбор)", "Неконтактные Суд", "Ожидание выезда", "Ожидание суда",
    "Подготовка к передаче в КА", "Преколлекшн (rename)", "Суд", "Усиление ПС Предцессия", 
    "Усиление позднего сбора", "ФЗ-230"
]
# Given column order
column_order = [
    "АО: Абонент не отвечает", "АО: Абонент недоступен", "АО: Номер не существует", 
    "АО: Нужен внутренний номер", "АО: Соединение установлено", "АО: Умные голосовые помощники", 
    "АО: Факс", "Должник молчит", "Должник не уверен", "Должник неизвестен", "Должник умер", 
    "Запрос реструктуризации", "Заявление о факте платежа", "Клиент болен", "Не передадут информацию", 
    "Не пройдена верификация", "Неконструктивный диалог", "Нужна помощь оператора", "Нужна помощь оператора *", 
    "Обещание оплатить", "Обещание частичной оплаты", "Оплата по испол.листу", "Отказ от верификации", 
    "Отказ от оплаты", "Отрицает долг", "Перезвонить", "Просьба передать информацию", "Сброс звонка роботом", 
    "Связь прервалась"
]

In [ ]:

# 1. Filter the dataframe
filtered_df = df[df['Результат автооценки'] != 100] # !!!
#filtered_df = df # !!!
# 2. Create the pivot table
pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='size', 
                                      fill_value=0)

'''pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='count', 
                                      fill_value=0)'''

pivot_table_mean = df.pivot_table(index='Имя колл-листа', columns='Результат робота', values='Результат автооценки', aggfunc='mean')


'''# 3. Ensure all desired columns are present
for col in column_order:
    if col not in pivot_table.columns:
        pivot_table[col] = 0

# Ensure all desired columns are present in the mean pivot table
for col in column_order:
    if col not in pivot_table_mean.columns:
        pivot_table_mean[col] = 0'''


'''# Reorder columns based on the given order
pivot_table = pivot_table[column_order]
pivot_table_mean = pivot_table_mean[column_order]
# Assuming pivot_table is the pivot table you previously created
pivot_table = pivot_table[column_order]
pivot_table = pivot_table.reindex(order)
pivot_table_mean = pivot_table_mean.reindex(order)'''

In [ ]:
pivot_table

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill
import xlsxwriter


def format_xlsx(pivot_table:pd.core.frame.DataFrame, sheet:str = 'Отчет 1', name:str = "pivot_table_gradient_colorscale.xlsx"):
    # Create an Excel writer and export the pivot table to an Excel file
    excel_file_path = name
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        pivot_table.to_excel(writer, sheet_name=sheet, index=True)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet]

        # Get the dimensions of the pivot table
        max_row = len(pivot_table)
        max_col = len(pivot_table.columns)

        # Add a format for the header cells
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'border': 1, 'bg_color': '#EFEFEF', 'align': 'center'})

        # Set the column width and format for the header
        for col_num, value in enumerate(pivot_table.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
            column_len = max(pivot_table[value].astype(str).str.len().max(), len(value)) + 2
            worksheet.set_column(col_num + 1, col_num + 1, column_len)
        
        # Add a format for the value cells
        value_format = workbook.add_format({'border': None, 'valign': 'vcenter', 'align': 'center',})

        # Get Max and Min of the value FFS
        max_value = pivot_table.max().max()
        min_value = pivot_table.min().min()


        # Apply gradient color scale to value cells
        worksheet.conditional_format(1, 1, max_row, max_col, {
            'type': '3_color_scale',
            'min_color': '#A6D86E',  # Green
            'mid_color': '#FFFFFE',  # White (for NaN)
            'max_color': '#e85f5f',  # Red
            'min_type': 'num',
            'mid_type': 'num',
            'max_type': 'num'
        })
        print(f'file: {name} -- Transformed 0')

In [ ]:
format_xlsx(pivot_table)

# Динамика по кол-листам + статусам + 3 в 1

Update RPC

In [360]:
import pandas as pd

csv_list = ['./Reports/universal_2.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]


In [361]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [362]:
# Mask
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [363]:

def update_index(dataframe):
    new_index = [f'{i if len(str(i)) > 1 else f"0{i}"} {row}' for i, row in enumerate(dataframe.index)]
    dataframe.index = new_index
    return dataframe
def run(df):
    # Create dynamic Mistakes count (1)
    pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
    pivot_df_mistakes = pivot_df_mistakes.fillna(0)
    pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
    pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
    pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
    tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
    pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
    pivot_df_mistakes = update_index(pivot_df_mistakes)
    # Create dynamic Calls count (2)
    pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
    pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
    pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
    tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
    pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
    pivot_df_calls = update_index(pivot_df_calls)
    # Create dynamic Mean Autoscore (3)
    pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
    pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
    pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
    pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
    pivot_df_mean = update_index(pivot_df_mean)
    # Create dynamic Error Percentage (4)
    pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
    pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
    pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
    pivot_df_error_rate = update_index(pivot_df_error_rate)
    # Create Mega-Pivot
    # Concatenate the pivot tables vertically along rows (axis=0)
    pivot_table = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
    pivot_table = pivot_table.sort_index()
    error_rate_rows = pivot_table[pivot_table.index.str.contains("(доля ошибок %)")]
    errors_rows = pivot_table[pivot_table.index.str.contains("(ошибки шт.)")]
    # Create a summary DataFrame with the desired column names
    # Find min/max percent
    max_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
    min_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()
    # Find min/max mistakes
    max_errors = errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
    min_errors = errors_rows.apply(pd.to_numeric, errors='coerce').min().min()
    # Returns thr pivot + kwargs
    return pivot_table, [max_percent, min_percent], [max_errors, min_errors]
# Create RPC-only frame
rpc_df = df[df['Контактное лицо'] == 'Должник']

pivot_all, percent, errors = run(df)
#del df
pivot_rpc, _, _ = run(rpc_df)
#del rpc_df

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [370]:
pivot_rpc.index[1]

'00 Актуализация (доля ошибок %)'

In [376]:
def update_rpc_index(pivot_rpc, pivot_all):
    # Create an empty dictionary to store index mapping
    index_mapping = {}
    # Iterate through the index of pivot_rpc
    for i in pivot_rpc.index:
        # Iterate through the index of pivot_all
        for j in pivot_all.index:
            # Check if the condition is met (i[3:] == j[3:])
            if i[3:] == j[3:]:
                # Update the index mapping
                index_mapping[i] = j
    # Use the rename method to update the index based on the mapping
    pivot_rpc = pivot_rpc.rename(index=index_mapping)
    return pivot_rpc
        

In [377]:
pivot_rpc.index

Index(['00 Актуализация (всего шт.)', '00 Актуализация (доля ошибок %)',
       '00 Актуализация (ошибки шт.)', '00 Актуализация (средняя АО)',
       '01 Внебаланс (new) (всего шт.)', '01 Внебаланс (new) (доля ошибок %)',
       '01 Внебаланс (new) (ошибки шт.)', '01 Внебаланс (new) (средняя АО)',
       '02 ДИЗ (всего шт.)', '02 ДИЗ (доля ошибок %)', '02 ДИЗ (ошибки шт.)',
       '02 ДИЗ (средняя АО)', '03 ДИЗ (возражения) (всего шт.)',
       '03 ДИЗ (возражения) (доля ошибок %)',
       '03 ДИЗ (возражения) (ошибки шт.)', '03 ДИЗ (возражения) (средняя АО)',
       '04 ДИЗ (чередование) (всего шт.)',
       '04 ДИЗ (чередование) (доля ошибок %)',
       '04 ДИЗ (чередование) (ошибки шт.)',
       '04 ДИЗ (чередование) (средняя АО)', '05 Ипотека (всего шт.)',
       '05 Ипотека (доля ошибок %)', '05 Ипотека (ошибки шт.)',
       '05 Ипотека (средняя АО)', '06 Исполнительное производство (всего шт.)',
       '06 Исполнительное производство (доля ошибок %)',
       '06 Исполнительное п

In [206]:
# Summary

def create_summary_pivot(pivot_all, pivot_rpc):
    def create_col(pivot, title, main_index):
        calls = pivot[pivot.index.str.contains('(всего шт.)')].apply(pd.to_numeric, errors='coerce')
        calls = calls.sum(axis=1)
        error_rate = pivot[pivot.index.str.contains('(доля ошибок %)')].apply(pd.to_numeric, errors='coerce')
        error_rate = error_rate.mean(axis=1,skipna=True,numeric_only=True)
        errors = pivot[pivot.index.str.contains('(ошибки шт.)')].apply(pd.to_numeric, errors='coerce')
        errors = errors.sum(axis=1)
        score = pivot[pivot.index.str.contains('(средняя АО)')].apply(pd.to_numeric, errors='coerce')
        score = score.mean(axis=1,skipna=True, numeric_only=True)
        summary = pd.DataFrame(index=main_index.index)
        summary[f'Сводные данные: {title}'] = pd.concat([calls, error_rate, errors, score], axis=0)
        summary = summary.sort_index()
        return summary
    full_summary = pd.DataFrame()
    full_summary = pd.concat([create_col(pivot_all, 'все звонки', main_index=pivot_all),
                              create_col(pivot_rpc, 'RPC', main_index=pivot_all) ], axis=1)
    return full_summary



In [207]:
create_summary_pivot(pivot_all,pivot_rpc).fillna('')

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Сводные данные: все звонки,Сводные данные: RPC
00 Актуализация (всего шт.),4670.000000,44
00 Актуализация (доля ошибок %),0.000323,0
00 Актуализация (ошибки шт.),3.000000,0
00 Актуализация (средняя АО),99.994071,100
01 Внебаланс (new) (всего шт.),14157.000000,202
...,...,...
24 Усиление ПС Предцессия (средняя АО),100.000000,
25 ФЗ-230 (всего шт.),13839.000000,
25 ФЗ-230 (доля ошибок %),0.000361,
25 ФЗ-230 (ошибки шт.),10.000000,


In [208]:
# Specify the Excel file path
name = 'output_new.xlsx'
excel_file_path = name
min_percent = percent[1]
max_percent = percent[0]
min_errors = errors[1]
max_errors = errors[0]
# Create a color scale conditional formatting rule
color_scale_rule_percent = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_percent,
                'mid_type': 'num',
                'mid_value': (max_percent-min_percent)/3,
                'max_type': 'num',
                'max_value': max_percent
                }

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/3,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    '''Function Start'''
    def create_sheet(pivot_table, sheet_name):
        # Write the DataFrame to the Excel file
        pivot_table.to_excel(writer, sheet_name=sheet_name)

        # Access the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]

        for row_num, row in enumerate(pivot_table.index):
            if "(доля ошибок %)" in row:
                format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
                worksheet.conditional_format(format_range, color_scale_rule_percent)
        # Define a percentage format
        percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

        for row_num, row in enumerate(pivot_table.index):
            if "(доля ошибок %)" in row:
                worksheet.set_row(row_num+1, None, percentage_format)

        for row_num, row in enumerate(pivot_table.index):
            if "(ошибки шт.)" in row:
                format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
                worksheet.conditional_format(format_range, color_scale_rule_errors)

        # Custom index format
        index_format = workbook.add_format(
            {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
        for row_num, row in enumerate(pivot_table.index):
            worksheet.write(f'A{row_num+2}', row, index_format)

        # Define a white fill format
        white_fill_format = workbook.add_format({'bg_color': '#FFFFFF', 'border': 0})

        # Apply the white background to the entire worksheet
        worksheet.set_column(0, 0, 60, white_fill_format)
        worksheet.set_column(1, 100, 18, white_fill_format)
    # Create 2 sheets
    create_sheet(pivot_all, 'Все звонки')
    create_sheet(pivot_rpc, 'RPC')
    # With = Save
print(f'file: {name} -- Transformed 0')


file: output_new.xlsx -- Transformed 0


In [ ]:
# Add numbers to index
# Add numbers to index
def update_index(dataframe):
    new_index = [f'{i if len(str(i)) > 1 else f"0{i}"} {row}' for i, row in enumerate(dataframe.index)]
    dataframe.index = new_index
    return dataframe


# Create dynamic Mistakes count (1)
pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
pivot_df_mistakes = pivot_df_mistakes.fillna(0)
pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
pivot_df_mistakes = update_index(pivot_df_mistakes)
# Create dynamic Calls count (2)
pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
pivot_df_calls = update_index(pivot_df_calls)
# Create dynamic Mean Autoscore (3)
pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
pivot_df_mean = update_index(pivot_df_mean)
# Create dynamic Error Percentage (4)
pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
pivot_df_error_rate = update_index(pivot_df_error_rate)
# Create Mega-Pivot
# Concatenate the pivot tables vertically along rows (axis=0)
mega_pivot = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
mega_pivot = mega_pivot.sort_index()


In [ ]:
error_rate_rows = mega_pivot[mega_pivot.index.str.contains("(доля ошибок %)")]
errors_rows = mega_pivot[mega_pivot.index.str.contains("(ошибки шт.)")]

In [ ]:
error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()

errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
errors_rows.apply(pd.to_numeric, errors='coerce').min().min()

Excel Shit

In [ ]:
import pandas as pd

# Find min/max percent
max_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Find min/max mistakes
max_errors = errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_errors = errors_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Specify the Excel file path
excel_file_path = 'output.xlsx'

# Create a color scale conditional formatting rule
color_scale_rule_percent = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_percent,
                'mid_type': 'num',
                'mid_value': (max_percent-min_percent)/3,
                'max_type': 'num',
                'max_value': max_percent
                }

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/3,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
mega_pivot.to_excel(writer, sheet_name='Все звонки')

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)

# Define a white fill format
white_fill_format = workbook.add_format({'bg_color': '#FFFFFF', 'border': 0})

# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 60, white_fill_format)
worksheet.set_column(1, 100, 18, white_fill_format)
'''# Custom header format
header_format = workbook.add_format(
                {'bold': True, 'border': 1, 'bg_color': '#AFEFEF','align': 'center'})
for col_num, value in enumerate(mega_pivot.columns.values):
                worksheet.write(0, col_num + 1, str(value)[:-19], header_format)'''
# Save the Excel file
writer.save()

# Отчет ошибки колл-лист + результат 

Pandas

In [270]:
import pandas as pd

csv_list = ['./Reports/universal_2.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])

In [271]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [356]:
def create_summary_pivot(df):
    def create_col(df, title='123', main_index=None):
        '''
        df: df main
        title: RPC?
        main_index: None id it's the main df, any pivot from the main df if it is RPC
        '''
        header_label = title
        pivot_df_calls = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Результат автооценки', aggfunc='count', fill_value='')
        # Calculate number of errors
        pivot_df_errors = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Ошибки', aggfunc='sum', fill_value='')
        # Calculate mean autoscore
        pivot_df_mean = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Результат автооценки', aggfunc='mean', fill_value='')
        # Calculate error rate
        pivot_df_error_rate = pd.DataFrame(pivot_df_errors['Ошибки'] / pivot_df_calls['Результат автооценки'])
        pivot_df_error_rate = pivot_df_error_rate.where(pivot_df_errors['Ошибки'] != 0, other='')
        # Create a DataFrame with 'Общий' as the main header
        header = pd.MultiIndex.from_tuples([(f'Срез: {header_label}', 'Ошб.%'), (f'Срез: {header_label}', 'Ошб.(шт.)'), (f'Срез: {header_label}', 'Зв.(шт.)'), (f'Срез: {header_label}', 'Ср.АО')])
        # Handle RPC Case
        if main_index is pd.DataFrame:
            summary = pd.DataFrame(columns=header, index=main_index.index)
        else:
            summary = pd.DataFrame(columns=header, index=pivot_df_error_rate.index)
        # Assign your Series to the corresponding columns
        summary[(f'Срез: {header_label}', 'Ошб.%')] = pivot_df_error_rate
        summary[(f'Срез: {header_label}', 'Ошб.(шт.)')] = pivot_df_errors
        summary[(f'Срез: {header_label}', 'Зв.(шт.)')] = pivot_df_calls
        summary[(f'Срез: {header_label}', 'Ср.АО')] = pivot_df_mean
        return summary, pivot_df_error_rate
    return create_col(df)

In [357]:
er, calls= create_summary_pivot(df)

In [359]:
er

Срез: 123                     \
                                                   Ошб.% Ошб.(шт.) Зв.(шт.)   
Имя колл-листа Результат робота                                               
Актуализация   АО: Абонент недоступен                            0      297   
               АО: Соединение установлено                        0        6   
               АО: Умные голосовые помощники                     0     1161   
               Должник молчит                                    0     2413   
               Должник неизвестен                                0       52   
...                                                  ...       ...      ...   
ФЗ-230         Отрицает долг                                     0        5   
               Перезвонить                                       0        9   
               Просьба передать информацию    0.00324675         1      308   
               Сброс звонка роботом                              0     1528   
               Связь прервалась                                  0      549   

                                                          
                                                   Ср.АО  
Имя колл-листа Результат робота                           
Актуализация   АО: Абонент недоступен         100.000000  
               АО: Соединение установлено     100.000000  
               АО: Умные голосовые помощники  100.000000  
               Должник молчит                 100.000000  
               Должник неизвестен             100.000000  
...                                                  ...  
ФЗ-230         Отрицает долг                  100.000000  
               Перезвонить                    100.000000  
               Просьба передать информацию     99.951299  
               Сброс звонка роботом           100.000000  
               Связь прервалась               100.000000  

[413 rows x 4 columns]

In [342]:
result = pd.DataFrame(erc['Ошибки'] / calls['Результат автооценки'])
result = result.where(erc['Ошибки'] != 0, other='')
result

0
Имя колл-листа Результат робота                         
Актуализация   АО: Абонент недоступен                   
               АО: Соединение установлено               
               АО: Умные голосовые помощники            
               Должник молчит                           
               Должник неизвестен                       
...                                                  ...
ФЗ-230         Отрицает долг                            
               Перезвонить                              
               Просьба передать информацию    0.00324675
               Сброс звонка роботом                     
               Связь прервалась                         

[413 rows x 1 columns]

In [346]:
calls

Результат автооценки
Имя колл-листа Результат робота                                   
Актуализация   АО: Абонент недоступен                          297
               АО: Соединение установлено                        6
               АО: Умные голосовые помощники                  1161
               Должник молчит                                 2413
               Должник неизвестен                               52
...                                                            ...
ФЗ-230         Отрицает долг                                     5
               Перезвонить                                       9
               Просьба передать информацию                     308
               Сброс звонка роботом                           1528
               Связь прервалась                                549

[413 rows x 1 columns]

In [291]:
# Create a DataFrame with 'Общий' as the main header
main_index = pivot_df_error_rate
header = pd.MultiIndex.from_tuples([('Общий', 'Ошб.%'), ('Общий', 'Ошб.(шт.)'), ('Общий', 'Зв.(шт.)'), ('Общий', 'Ср.АО')])
# Create an empty DataFrame with the specified header
summary = pd.DataFrame(columns=header, index=pivot_df_error_rate.index)

# Assign your Series to the corresponding columns
summary[('Общий', 'Ошб.%')] = pivot_df_error_rate
summary[('Общий', 'Ошб.(шт.)')] = pivot_df_errors
summary[('Общий', 'Зв.(шт.)')] = pivot_df_calls
summary[('Общий', 'Ср.АО')] = pivot_df_mean


In [292]:
summary

Общий                     \
                                             Ошб.% Ошб.(шт.) Зв.(шт.)   
Имя колл-листа Результат робота                                         
Актуализация   АО: Абонент недоступен                      0      297   
               АО: Соединение установлено                  0        6   
               АО: Умные голосовые помощники               0     1161   
               Должник молчит                              0     2413   
               Должник неизвестен                          0       52   
...                                            ...       ...      ...   
ФЗ-230         Отрицает долг                               0        5   
               Перезвонить                                 0        9   
               Просьба передать информацию                 1      308   
               Сброс звонка роботом                        0     1528   
               Связь прервалась                            0      549   

                                                          
                                                   Ср.АО  
Имя колл-листа Результат робота                           
Актуализация   АО: Абонент недоступен         100.000000  
               АО: Соединение установлено     100.000000  
               АО: Умные голосовые помощники  100.000000  
               Должник молчит                 100.000000  
               Должник неизвестен             100.000000  
...                                                  ...  
ФЗ-230         Отрицает долг                  100.000000  
               Перезвонить                    100.000000  
               Просьба передать информацию     99.951299  
               Сброс звонка роботом           100.000000  
               Связь прервалась               100.000000  

[413 rows x 4 columns]

In [274]:
def create_multiindex(dataframe, sub_index:str):
    # Create MultiIndex
    multiindex = []
    for i, column in enumerate(dataframe):
        multiindex.append((column, sub_index))
    dataframe.columns = pd.MultiIndex.from_tuples(multiindex)
    return dataframe

def create_summary_pivot(df):
    def create_col(df, title, main_index):
    # Calculate number of calls for each pair
        pivot_df_calls = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Результат автооценки', aggfunc='count', fill_value='')
        # Calculate number of errors
        pivot_df_errors = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Ошибки', aggfunc='sum', fill_value='')
        # Calculate mean autoscore
        pivot_df_mean = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], values='Результат автооценки', aggfunc='mean', fill_value='')
        # Calculate error rate
        pivot_df_error_rate = (pivot_df_errors.replace("", pd.NA) / pivot_df_calls.replace("", pd.NA)).applymap(lambda x: x if not pd.isna(x) else '')
        summary = pd.DataFrame()
        summary[f'Свод: {title}'] = pd.concat([pivot_df_error_rate, pivot_df_errors, pivot_df_calls, pivot_df_mean], axis=1)
        summary = summary.sort_index()
        return summary
    rpc_df = df[df['Контактное лицо'] == 'Должник']
    summary_all = create_col(df, 'Срез: все звонки', df)
    summary_rpc = create_col(rpc_df, 'Срез: RPC', df)
    return summary_all, summary_rpc



# Calculate number of calls for each pair
pivot_df_calls = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='count', fill_value='')
pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
# Calculate number of errors
pivot_df_errors = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum', fill_value='')
pivot_df_errors.columns = pd.to_datetime(pivot_df_errors.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_errors = pivot_df_errors.sort_index(axis=1)  # Fix date Time
# Calculate mean autoscore
pivot_df_mean = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
# Calculate error rate
pivot_df_error_rate = (pivot_df_errors.replace("", pd.NA) / pivot_df_calls.replace("", pd.NA)).applymap(lambda x: x if not pd.isna(x) else '')
pivot_df_error_rate.columns = pd.to_datetime(pivot_df_error_rate.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_error_rate = pivot_df_error_rate.sort_index(axis=1)  # Fix date Time
max_error_rate = pivot_df_error_rate.apply(pd.to_numeric, errors='coerce').max().max()
min_error_rate = pivot_df_error_rate.apply(pd.to_numeric, errors='coerce').min().min()
# Create MultiIndex
pivot_df_calls = create_multiindex(pivot_df_calls, 'Зв.(шт.)')
pivot_df_errors = create_multiindex(pivot_df_errors, 'Ошб.(шт.)')
pivot_df_mean = create_multiindex(pivot_df_mean, 'Ср.АО')
pivot_df_error_rate = create_multiindex(pivot_df_error_rate, 'Ошб.%')

In [275]:
i, j = create_summary_pivot(df)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [265]:
# Create a list of the DataFrames you want to merge
dfs_to_merge = [pivot_df_calls, pivot_df_errors, pivot_df_mean, pivot_df_error_rate]

# Initialize an empty DataFrame with the same index as the original DataFrames
merged_df = pd.DataFrame(index=pivot_df_calls.index)

'''CREATE MULTIINDEX'''
multi_index = []
# Iterate through the DataFrames and concatenate their columns in the desired order
for num, column in enumerate(pivot_df_calls.columns):
        for dataframe in dfs_to_merge:
                #print(dataframe.iloc[:, num].name)
                #merged_df[dataframe.iloc[:, num].]
                col_name = (dataframe.iloc[:, num].name[0], dataframe.iloc[:, num].name[1])
                # Append the column name tuple to the list
                multi_index.append(col_name)
                merged_df[col_name] = dataframe.iloc[:, num]

merged_df.columns = pd.MultiIndex.from_tuples(multi_index)


In [267]:
merged_df

2023-07-29                     \
                                               Зв.(шт.) Ошб.(шт.)    Ср.АО   
Имя колл-листа Результат робота                                              
Актуализация   АО: Абонент недоступен                66         0      100   
               АО: Номер не существует                1         0      100   
               АО: Нужен внутренний номер             7         0      100   
               АО: Соединение установлено            13         0      100   
               АО: Умные голосовые помощники        487         2  99.9384   
...                                                 ...       ...      ...   
ФЗ-230         Отрицает долг                         82         7  97.8659   
               Перезвонить                          221         7  99.2534   
               Просьба передать информацию         1743        37  99.5582   
               Сброс звонка роботом                5645         0      100   
               Связь прервалась                    9292         2  99.9968   

                                                          2023-07-30  \
                                                    Ошб.%   Зв.(шт.)   
Имя колл-листа Результат робота                                        
Актуализация   АО: Абонент недоступен                   0         57   
               АО: Номер не существует                  0              
               АО: Нужен внутренний номер               0         14   
               АО: Соединение установлено               0          3   
               АО: Умные голосовые помощники   0.00410678        461   
...                                                   ...        ...   
ФЗ-230         Отрицает долг                    0.0853659         68   
               Перезвонить                      0.0316742        220   
               Просьба передать информацию      0.0212278       1897   
               Сброс звонка роботом                     0       5808   
               Связь прервалась               0.000215239      10309   

                                                                              \
                                             Ошб.(шт.)    Ср.АО        Ошб.%   
Имя колл-листа Результат робота                                                
Актуализация   АО: Абонент недоступен                0      100            0   
               АО: Номер не существует                                         
               АО: Нужен внутренний номер            0      100            0   
               АО: Соединение установлено            0      100            0   
               АО: Умные голосовые помощники         0      100            0   
...                                                ...      ...          ...   
ФЗ-230         Отрицает долг                        12  95.5882     0.176471   
               Перезвонить                           8  99.1364    0.0363636   
               Просьба передать информацию          45  99.4992    0.0237217   
               Сброс звонка роботом                  0      100            0   
               Связь прервалась                      3  99.9947  0.000291008   

                                             2023-08-01            ...  \
                                               Зв.(шт.) Ошб.(шт.)  ...   
Имя колл-листа Результат робота                                    ...   
Актуализация   АО: Абонент недоступен                 6         0  ...   
               АО: Номер не существует                             ...   
               АО: Нужен внутренний номер                          ...   
               АО: Соединение установлено            21         0  ...   
               АО: Умные голосовые помощники        212         1  ...   
...                                                 ...       ...  ...   
ФЗ-230         Отрицает долг                                       ...   
               Перезвонить                                         ...   
        

Excel

In [261]:
max_errors = max_error_rate  # kwarg
min_errors = min_error_rate  # kwarg
pivot_table = merged_df
"""!!!"""
pivot_table = pivot_table.reset_index()
name = 'Отчет 1'
sheet = 'Все звонки'

# Specify the Excel file path
excel_file_path = f'{name}.xlsx'

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/4,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
pivot_table.to_excel(writer, sheet_name=sheet)

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

'''for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)
'''
# Define a white fill format
white_fill_format = workbook.add_format({'text_wrap': True, 'bg_color': '#FFFFFF', 'border': 0})
white_index_format = workbook.add_format({'text_wrap': True,
                                          'bg_color': '#FFFFFF',
                                          'border': 0,
                                          'bold': True
                                          })
# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 5, white_fill_format)
worksheet.set_column(1, 1, 25, white_index_format)
worksheet.set_column(2, 2, 30, white_index_format)
worksheet.set_column(3, 100, 9, white_fill_format)

percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF', 'border': 0})

for i, j in enumerate(pivot_table.head()):
    if j[1] == 'Ошб.%':
        worksheet.set_column(i+1, i+1, None, percentage_format)
        worksheet.conditional_format(0, i+1, 999, i+1, color_scale_rule_errors)

# Save the Excel file
writer.save()
    

# Страшный отчет: по блокам

Pandas

In [1]:
# 10.8 GB --> 20 GB --> 15 GB
import numpy as np
import pandas as pd

csv_list = ['./Reports/block_test.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

'''
№ п/п;ID звонка;Имя колл-листа;Результат робота;Дата звонка;Результат автооценки;Поисковый запрос: Ошибка идентификации: не уточняет дату рождения
'''

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
#df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

# GET HEADERS AND REMOVE IRRELEVANT ONES
queries_list = list(df.columns)
columns_to_remove = [
    '№ п/п',
    'ID звонка',
    'Имя колл-листа',
    'Результат робота',
    'Дата звонка',
    'Результат автооценки',
    'Ошибки',
    'Дата',
    'Поисковый запрос: Все звонки, балл'
]
# Remove the specified columns
queries_list = [col for col in queries_list if col not in columns_to_remove]

# Все звонки fix
df['Всего звонков по листу'] = df['Поисковый запрос: Все звонки, балл']
queries_list.append('Всего звонков по листу')
#df['Всего звонков по листу'] = df['Всего звонков по листу'] * -1

for col in queries_list:
    df[col] = df[col] / df[col]

queries_list.append('Ошибки')

In [4]:
print(df.iloc[0:1])


   № п/п                               ID звонка        Имя колл-листа  \
0    1.0  rsb-1-1-1694307634453184577.5667523458  Коллекшн ранний сбор   

                Результат робота          Дата звонка  Результат автооценки  \
0  АО: Умные голосовые помощники  10.09.2023 04:00:39                 100.0   

   Поисковый запрос: Ошибка идентификации: не уточняет дату рождения, балл  \
0                                                NaN                         

   Поисковый запрос: Спрашивает дату, когда не нужно, балл  \
0                                                NaN         

   Поисковый запрос: Перебивает и не даёт сказать дату до конца, балл  \
0                                                NaN                    

   Поисковый запрос: Разглашение БТ, балл  ...  \
0                                     NaN  ...   

   Поисковый запрос: Неправомерный отказ 1, балл  \
0                                            NaN   

   Поисковый запрос: Неправильная активность в АС Калита 

In [25]:
# First, melt the DataFrame to convert 'Запрос_1', 'Запрос_2', 'Запрос_3' into rows
melted_df = pd.melt(df, id_vars=['Имя колл-листа', 'Дата'], value_vars = queries_list, var_name='Запрос', value_name='Ошибки шт.')
melted_df.reset_index(drop=True)
'''I could insert check here that will assign a block based on Имя кол-лист, but it will decrease the time'''
del df

In [26]:

# Replace -1 with 1
#melted_df['Ошибки шт.'] = melted_df['Ошибки шт.'] * -1

# Now, create a pivot table to calculate sums
pivot_table = melted_df.pivot_table(
    values='Ошибки шт.',
    index=['Имя колл-листа', 'Запрос'],
    columns='Дата',
    aggfunc='sum'
)
del melted_df
# If you want to reset the index and have a cleaner view
pivot_table.reset_index()



Дата,Имя колл-листа,Запрос,10.09.2023,11.09.2023,12.09.2023,13.09.2023
0,Актуализация,Всего звонков по листу,7018,4969,5554,7167
1,Актуализация,Ошибки,1,4,4,3
2,Актуализация,"Поисковый запрос: Выслушайте меня, балл",0,0,0,0
3,Актуализация,"Поисковый запрос: Выявление тишины, балл",0,0,0,0
4,Актуализация,Поисковый запрос: Диалог с автоответчиком > 30...,1,2,1,0
...,...,...,...,...,...,...
1395,ФЗ-230,Поисковый запрос: Робот некорректно фиксирует ...,0,0,0,0
1396,ФЗ-230,Поисковый запрос: Робот некорректно фиксирует ...,0,0,0,0
1397,ФЗ-230,"Поисковый запрос: Спрашивает дату, когда не ну...",0,0,0,0
1398,ФЗ-230,Поисковый запрос: вместо тел не принадлежит - ...,0,0,1,0


Excel

In [28]:
pivot_table = pivot_table.iloc[:, 1:]
name = 'Отчет 4'
sheet = 'Все звонки'

# Specify the Excel file path
excel_file_path = f'{name}.xlsx'

# Create a color scale conditional formatting rule
'''color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/4,
                'max_type': 'num',
                'max_value': max_errors
                }'''

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
pivot_table.to_excel(writer, sheet_name=sheet)

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

'''for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)
'''
# Define a white fill format
white_fill_format = workbook.add_format({'text_wrap': True, 'bg_color': '#FFFFFF', 'border': 0})
# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 40, white_fill_format)
worksheet.set_column(1, 1, 80, white_fill_format)
worksheet.set_column(0, 100, 20, white_fill_format)

'''percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF', 'border': 0})

for i, j in enumerate(pivot_table.head()):
    if j[1] == 'Ошб.%':
        worksheet.set_column(i+2, i+2, None, percentage_format)
        worksheet.conditional_format(0, i+2, 999, i+2, color_scale_rule_errors)'''

# Save the Excel file
writer.save()

In [ ]:
worksheet.col

# Отчет Блок

Pandas

import

In [21]:
import pandas as pd

def prep_data(csv_list):

    # Concatenate all csv to a single biiig df
    df = pd.DataFrame()
    for i in csv_list:
        df_add = pd.read_csv(i, sep=';', encoding='utf-8',header=0)
        df = pd.concat([df, df_add], ignore_index=True)
    # Fix мультидоговоры for RSB
    mask = df['№ п/п'].isna()
    df = df[~mask]
    # Convert the 'Длительность звонка' column to Timedelta
    df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
    df['Ошибки'] = df['Результат автооценки'] != 100
    #df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
    # Try this
    df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
    df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
    df = df.reset_index(drop=True)
    return df

def create_pivot(df, rpc=False):
    # Add numbers to index
    def update_index(dataframe):
        new_index = [f'{i if len(str(i)) > 1 else f"0{i}"} {row}' for i, row in enumerate(dataframe.index)]
        dataframe.index = new_index
        return dataframe
    print(1)
    # Create dynamic Mistakes count (1)
    pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
    print(2)
    pivot_df_mistakes = pivot_df_mistakes.fillna(0)
    pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
    pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
    print(2)
    pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
    tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
    pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
    pivot_df_mistakes = update_index(pivot_df_mistakes)
    # Create dynamic Calls count (2)
    print(3)
    pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
    pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
    pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
    tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
    pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
    pivot_df_calls = update_index(pivot_df_calls)
    print(4)
    # Create dynamic Mean Autoscore (3)
    pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
    pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
    pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
    pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
    pivot_df_mean = update_index(pivot_df_mean)
    print(5)
    # Create dynamic Error Percentage (4)
    pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
    pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
    pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
    pivot_df_error_rate = update_index(pivot_df_error_rate)
    print(6)
    # Create Mega-Pivot
    # Concatenate the pivot tables vertically along rows (axis=0)
    pivot_table = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
    pivot_table = pivot_table.sort_index()
    error_rate_rows = pivot_table[pivot_table.index.str.contains("(доля ошибок %)")]
    errors_rows = pivot_table[pivot_table.index.str.contains("(ошибки шт.)")]
    print(7)
    # Find min/max percent
    max_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
    min_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()
    # Find min/max mistakes
    max_errors = errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
    min_errors = errors_rows.apply(pd.to_numeric, errors='coerce').min().min()
    # Returns thr pivot + kwargs
    return pivot_table, [max_percent, min_percent], [max_errors, min_errors]

'''Excel Code'''
def format_xlsx(pivot_table: pd.DataFrame,
                name: str = "pivot_table_2_call_lists.xlsx", **kwargs):
    print(1)
    # Specify the Excel file path
    excel_file_path = name
    min_percent = kwargs['percent'][1]
    max_percent = kwargs['percent'][0]
    min_errors = kwargs['errors'][1]
    max_errors = kwargs['errors'][0]
    print
    # Create a color scale conditional formatting rule
    color_scale_rule_percent = {
                    'type': '3_color_scale',
                    'min_color': '#A6D86E',  # Green
                    'mid_color': '#FCFAA0',  # White (for NaN)
                    'max_color': '#e85f5f',  # Red
                    'min_type': 'num',
                    'min_value': min_percent,
                    'mid_type': 'num',
                    'mid_value': (max_percent-min_percent)/3,
                    'max_type': 'num',
                    'max_value': max_percent
                    }

    # Create a color scale conditional formatting rule
    color_scale_rule_errors = {
                    'type': '3_color_scale',
                    'min_color': '#A6D86E',  # Green
                    'mid_color': '#FCFAA0',  # White (for NaN)
                    'max_color': '#e85f5f',  # Red
                    'min_type': 'num',
                    'min_value': min_errors,
                    'mid_type': 'num',
                    'mid_value': (max_errors-min_errors)/3,
                    'max_type': 'num',
                    'max_value': max_errors
                    }

    # Create a Pandas Excel writer using xlsxwriter as the engine
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        '''Function Start'''
        # Write the DataFrame to the Excel file
        pivot_table.to_excel(writer, sheet_name='Все звонки')
        print(1)
        # Access the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Все звонки']
        print(1)
        for row_num, row in enumerate(pivot_table.index):
            if "(доля ошибок %)" in row:
                format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
                worksheet.conditional_format(format_range, color_scale_rule_percent)
        # Define a percentage format
        percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})
        print(1)
        for row_num, row in enumerate(pivot_table.index):
            if "(доля ошибок %)" in row:
                worksheet.set_row(row_num+1, None, percentage_format)

        for row_num, row in enumerate(pivot_table.index):
            if "(ошибки шт.)" in row:
                format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
                worksheet.conditional_format(format_range, color_scale_rule_errors)
        print(1)
        # Custom index format
        index_format = workbook.add_format(
            {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
        for row_num, row in enumerate(pivot_table.index):
            worksheet.write(f'A{row_num+2}', row, index_format)

        # Define a white fill format
        white_fill_format = workbook.add_format({'bg_color': '#FFFFFF', 'border': 0})
        print(1)
        # Apply the white background to the entire worksheet
        worksheet.set_column(0, 0, 60, white_fill_format)
        worksheet.set_column(1, 100, 18, white_fill_format)
        # With = Save
        worksheet.close()
    print(f'file: {name} -- Transformed 0')

In [22]:
i = prep_data(['./Reports/08-08.csv'])

In [23]:
df, percent, errors = create_pivot(i, rpc=False)

1
2
2
3
4
5
6
7


c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
